In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import pickle
import time
from gensim.models.doc2vec import Doc2Vec
from gensim.models import KeyedVectors

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [3]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

In [4]:
df=pd.read_hdf('df.h5','table')

FileNotFoundError: File /home/ubuntu/hana/df.h5 does not exist

In [5]:
df[df['style'].notnull()].shape

NameError: name 'df' is not defined

In [5]:
doc2vec_model = Doc2Vec.load('model/doc2vec/model_dv')
doc_vectors = KeyedVectors.load('model/doc2vec/dv_vectors')

In [6]:
desc = df['product_id'].apply(lambda row: doc2vec_model.docvecs[row])
labels = df['style']

In [11]:
x_train, x_test= desc[df['style'].notnull()], desc[df['style'].isnull()]
y_train, y_test= labels[df['style'].notnull()], labels[df['style'].isnull()]
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2018)
x_train, x_val, x_test = np.vstack(x_train), np.vstack(x_val), np.vstack(x_test)

In [19]:
# lb = preprocessing.Label()
# lb.fit(labels[df['style'].notnull()])
# y_train, y_val=lb.transform(y_train), lb.transform(y_val)
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((67859, 50), (16965, 50), (987326, 50), (67859,), (16965,), (987326,))

In [24]:
from sklearn.metrics import classification_report
from sklearn.metrics import zero_one_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import jaccard_similarity_score

def custom_metrics(y_true,y_pred):
    print('hamming: {:.3f}\n0-1: {:.3f}\njaccard: {:.3f}'.
         format(hamming_loss(y_true,y_pred),
               zero_one_loss(y_true,y_pred),
               jaccard_similarity_score(y_true,y_pred)))
    print(classification_report(y_true,y_pred))

In [40]:
from sklearn.linear_model import LogisticRegressionCV

In [211]:
%%time
classifierCV = LogisticRegressionCV(Cs=list(np.power(10.0, np.arange(-10, 10)))
                                    ,penalty='l2'
                                    ,scoring='accuracy'
                                    ,cv=10
                                    ,random_state=2018
                                    ,max_iter=10000
                                    ,fit_intercept=True
                                    ,solver='newton-cg'
                                    ,tol=10)
# classifier = LogisticRegressionCV(cv=10, random_state=2018, penalty='l2', solver='lbfgs', multi_class='multinomial')
classifierCV.fit(x_train, y_train)

CPU times: user 16min 24s, sys: 29min 53s, total: 46min 17s
Wall time: 3min 15s


In [212]:
predictions_prob=classifierCV.predict_proba(x_val)
predictions=classifierCV.predict(x_val)

In [272]:
np.mean(predictions==y_val)

0.30810492189802535

In [213]:
pred_df=pd.concat([pd.DataFrame([classifierCV.classes_.tolist()]), pd.DataFrame(predictions_prob)])
pred_df=pred_df.rename(columns=pred_df.iloc[0], inplace=False).reset_index(drop=True)
pred_df.drop(index=[0], axis=1, inplace=True)
pred_df.reset_index(drop=True, inplace=True)
pred_df['pred_style']=predictions

In [214]:
df_merge=df.loc[y_val.index][['style', 'product_id', 'comb_desc']].reset_index()

In [215]:
pred_df=pd.concat([pred_df, df_merge], axis=1)

for each product_id, keep only the pred prob larger than 0.1, and label it, remove the product_id that all styles has <0.1 probs. go back to cluster.

In [266]:
styles = pred_df.columns[:-5].tolist()

def topProbability(row, threshold=0.1):
    return [s for s in styles if row[s]>threshold]

In [267]:
top_styles=pred_df.apply(topProbability, axis=1)

In [268]:
top_styles

0        [bohemian, gothic, minimalistic, romantic]                     
1        [bohemian, gothic, minimalistic, tailored]                     
2        [bohemian, minimalistic, sartorial]                            
3        [active, bohemian, minimalistic, sartorial]                    
4        [active, minimalistic]                                         
5        [bohemian, classic, romantic]                                  
6        [bohemian, minimalistic, tailored]                             
7        [active, bohemian, minimalistic, sartorial]                    
8        [active, classic, edgy, gothic]                                
9        [bohemian, classic, sartorial]                                 
10       [bohemian, minimalistic]                                       
11       [bohemian, gothic, minimalistic]                               
12       [bohemian, minimalistic, preppy, romantic, sartorial, tailored]
13       [active, bohemian, classic, sartorial]    

In [1]:
pred_df.head()

NameError: name 'pred_df' is not defined

In [232]:
# save the model to disk
model_path= 'model/multiclass_classification/lrcv/'
filename = 'lgcv.sav'
pickle.dump(classifierCV, open(model_path+filename, 'wb'))
# classifierCV = pickle.load(open(filename, 'rb'))

In [233]:
pred_df.to_hdf(model_path+'pred_df.h5','table')

/home/ubuntu/.virtualenvs/hana/lib/python3.6/site-packages/pandas/core/generic.py:1993: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['active', 'athleisure', 'bohemian', 'classic', 'edgy', 'glam', 'gothic', 'hipster', 'minimalistic', 'preppy', 'punk', 'romantic', 'sartorial', 'tailored', 'vintage', 'western', 'pred_style', 'style', 'product_id', 'comb_desc']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [34]:
custom_metrics(y_val, predictions)

hamming: 0.629
0-1: 0.629
jaccard: 0.371
              precision    recall  f1-score   support

      active       0.37      0.35      0.36      1535
  athleisure       0.00      0.00      0.00        62
    bohemian       0.39      0.61      0.48      1898
     classic       0.39      0.36      0.37      1584
        edgy       0.35      0.22      0.27       944
        glam       0.25      0.04      0.06       352
      gothic       0.35      0.44      0.39      1478
     hipster       0.73      0.17      0.27        66
minimalistic       0.37      0.50      0.42      2370
      preppy       0.29      0.18      0.22      1492
        punk       0.33      0.04      0.06       425
    romantic       0.39      0.16      0.23      1010
   sartorial       0.40      0.46      0.43      1899
    tailored       0.36      0.43      0.39      1454
     vintage       0.34      0.11      0.16       323
     western       0.00      0.00      0.00        73

 avg / total       0.36      0.37      

In [ ]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above
allDone()

In [270]:
df.sample(10)

,product_id,designer_original_name,product_name,comb_desc,From,style
884308,687720652,ronny kobo,Ronny Kobo Women's Ava Eyelets & Stripes Bodysuit,ava knit eyelet asymmetrical neck shoulder sleeve tonal panel seaming fully line circumference bust shoulder hem take size height tops,shopstyle,NaN
846106,365338508,5.11 tactical,Men's 5.11 Tactical Short Sleeve Tall Professional Polo - Heather Grey Short Sleeve Shirts,short sleeve tall professional polo heather grey short sleeve law enforcement military consistently choose professional polo wear meet law enforcement professional polo stay look neat clean professional duty assignment professional polo select casual uniform wear country cotton specially fade wrinkle shrink professional polo provide neat clean professional look professional polo wear duty professional polo perfect special bicycle patrol casual duty work training facility wear office wear duty demand formal uniform professional polo high law enforcement military roll collar flexible stay roll pen endure constant use design comfort durability melamine button professional polo melamine button crack break withstand heat iron professional look stitching construction professional polo high quality professional polo ultimate comfort fit cotton fabric soft feel look great polo,shopstyle,NaN
973594,5af809b64500d65f249e13fc,derek lam 10 crosby,Long Sleeve Asymmetrical Ruffle Hem Dress,long sleeve asymmetrical ruffle hem dress shift dress asymmetric ruffle hem white silk weave metallic gold invisible zip closure clean crew neck button line white soft white clothing,farfetch,None
512034,703075258,dondup,Dondup Sun Viscose Blend T-shirt,sun viscose blend sun viscose blend round neck elastic hem metal panel viscose blue,shopstyle,NaN
582535,707569679,roberto collina,ROBERTO COLLINA Sweaters,knit solid color round collar lightweight sweater long viscose linen nylon beige,shopstyle,NaN
861616,654778344,apiece apart,Apiece Apart off the shoulder blouse,shoulder blouse grey blue cotton blouse shoulder design frill trim curved hem long left sleeve button cuff blue top,shopstyle,NaN
418716,700504297,pottery barn kids,Mackenzie Navy Mermaids Large Duffle Bag,navy large bag big big bag extra roomy bag easy spacious overnight bag polyester detachable shoulder strap rolling build withstand matter rugged denier polyester interior denier polyester shoulder strap durable synthetic webbing roll carry carry strap easy transport load heavy key product shoulder strap adjustable roomy interior store personalization note personalization placement appear show personalization cover print roll overall wide deep high interior wide deep high exterior pocket wide long carrying handle wide long carry strap wide long diameter weight capacity weight care maintenance spot clean damp white cloth blot remove excess water air dry nursery clothe,shopstyle,NaN
502406,675654787,aviu,AVIÙ T-shirts,tulle jersey openwork solid color round collar long stretch cotton polyamide black,shopstyle,NaN
586817,700073142,lord & taylor,Lord & Taylor Petite Embroidered Ankle Pants,petite ankle pant petite ankle ankle pant allover floral embroidery belt zip fly button closure style skinny fit rise inseam machine wash authentic navy petite,shopstyle,NaN
461886,692606242,parker,Parker Honey Cold Shoulder Blouse,honey cold shoulder blouse velvet tie narrow add festive cut airy silk beguilingly style honey cold shoulder blouse style number available black top,shopstyle,NaN
